In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
data=pd.read_excel('赛题1数据材料\赛题1数据集\上市信息财务信息-利润表.xlsx')

In [3]:
data

,企业编号,标题,日期,营业收入(元),营业成本(元),销售费用(元),财务费用(元),管理费用(元),资产减值损失(元),投资收益(元),营业利润(元),利润总额(元),所得税(元),归属母公司所有者净利润(元)
0,1001,利润表,2018-09-30,127109000000,NaN,NaN,NaN,30421000000,44072000000,11116000000,50918000000,50893000000,7229000000,43034000000
1,1001,利润表,2018-06-30,82256000000,NaN,NaN,NaN,19971000000,26929000000,9382000000,34164000000,34161000000,5261000000,28432000000
2,1001,利润表,2018-03-31,39629000000,NaN,NaN,NaN,9680000000,12173000000,3696000000,17105000000,17106000000,2647000000,14266000000
3,1001,利润表,2017-12-31,168619000000,NaN,NaN,NaN,41047000000,55285000000,13585000000,70275000000,69828000000,14826000000,54046000000
4,1001,利润表,2017-09-30,125471000000,NaN,NaN,NaN,29275000000,40145000000,8391000000,54621000000,54743000000,12347000000,41305000000
5,1001,利润表,2017-06-30,84045000000,NaN,NaN,NaN,19236000000,27170000000,6138000000,36671000000,36751000000,8229000000,27618000000
6,1002,利润表,2018-09-30,5803394924.71,4145927467.38,21521618.9,72485135.28,337278554.46,11890361.38,35859554.47,1247170227.63,1252524462.81,312893396.74,914633402.37
7,1002,利润表,2018-06-30,3720615775.06,2524410387.14,16058592.17,32046137.95,188584639.56,31814002.69,17785221.48,951928844.35,954963763.65,239172814.25,700857214.35
8,1002,利润表,2018-03-31,1723798620.17,1039236420.46,13675219.71,-5158955.5,67359452.91,11965766.7,8784291.9,594946239.92,596388971.82,159952657.62,431843540.39
9,1002,利润表,2017-12-31,6761550874.37,4018885671.11,88346033.54,38668629.07,423106162.89,18778746.5,45618283.41,2164936622.04,2165782070.2,564537893.88,1564238738.86


In [4]:
#将单位全部统一为以元为单位
def convert(cell):
    pattern1=(r'(-?\d+\.?\d+)亿')
    pattern2=(r'(-?\d+\.?\d+)万亿')
    pattern3=(r'(-?\d+\.?\d+)万')
    if re.findall(pattern1,cell)!=[]:
        cell=float(re.findall(pattern1, cell)[0])*100000000
    elif re.findall(pattern2,cell)!=[]:
        cell=float(re.findall(pattern2, cell)[0])*1000000000000
    elif re.findall(pattern3,cell)!=[]:
        cell=float(re.findall(pattern3, cell)[0])*10000
    elif cell=='nan'or cell=='--':
        cell=np.nan
    else: cell=float(cell)
    return cell

In [5]:
#先将所有列转为str然后再进行匹配
data['营业收入(元)']=data['营业收入(元)'].apply(str).map(convert)
data['营业成本(元)']=data['营业成本(元)'].apply(str).map(convert)
data['销售费用(元)']=data['销售费用(元)'].apply(str).map(convert)
data['财务费用(元)']=data['财务费用(元)'].apply(str).map(convert)
data['管理费用(元)']=data['管理费用(元)'].apply(str).map(convert)
data['资产减值损失(元)']=data['资产减值损失(元)'].apply(str).map(convert)
data['投资收益(元)']=data['投资收益(元)'].apply(str).map(convert)
data['营业利润(元)']=data['营业利润(元)'].apply(str).map(convert)
data['利润总额(元)']=data['利润总额(元)'].apply(str).map(convert)
data['所得税(元)']=data['所得税(元)'].apply(str).map(convert)
data['归属母公司所有者净利润(元)']=data['归属母公司所有者净利润(元)'].apply(str).map(convert)

In [6]:
data

,企业编号,标题,日期,营业收入(元),营业成本(元),销售费用(元),财务费用(元),管理费用(元),资产减值损失(元),投资收益(元),营业利润(元),利润总额(元),所得税(元),归属母公司所有者净利润(元)
0,1001,利润表,2018-09-30,1.271090e+11,NaN,NaN,NaN,3.042100e+10,4.407200e+10,1.111600e+10,5.091800e+10,5.089300e+10,7.229000e+09,4.303400e+10
1,1001,利润表,2018-06-30,8.225600e+10,NaN,NaN,NaN,1.997100e+10,2.692900e+10,9.382000e+09,3.416400e+10,3.416100e+10,5.261000e+09,2.843200e+10
2,1001,利润表,2018-03-31,3.962900e+10,NaN,NaN,NaN,9.680000e+09,1.217300e+10,3.696000e+09,1.710500e+10,1.710600e+10,2.647000e+09,1.426600e+10
3,1001,利润表,2017-12-31,1.686190e+11,NaN,NaN,NaN,4.104700e+10,5.528500e+10,1.358500e+10,7.027500e+10,6.982800e+10,1.482600e+10,5.404600e+10
4,1001,利润表,2017-09-30,1.254710e+11,NaN,NaN,NaN,2.927500e+10,4.014500e+10,8.391000e+09,5.462100e+10,5.474300e+10,1.234700e+10,4.130500e+10
5,1001,利润表,2017-06-30,8.404500e+10,NaN,NaN,NaN,1.923600e+10,2.717000e+10,6.138000e+09,3.667100e+10,3.675100e+10,8.229000e+09,2.761800e+10
6,1002,利润表,2018-09-30,5.803395e+09,4.145927e+09,2.152162e+07,7.248514e+07,3.372786e+08,1.189036e+07,3.585955e+07,1.247170e+09,1.252524e+09,3.128934e+08,9.146334e+08
7,1002,利润表,2018-06-30,3.720616e+09,2.524410e+09,1.605859e+07,3.204614e+07,1.885846e+08,3.181400e+07,1.778522e+07,9.519288e+08,9.549638e+08,2.391728e+08,7.008572e+08
8,1002,利润表,2018-03-31,1.723799e+09,1.039236e+09,1.367522e+07,-5.158956e+06,6.735945e+07,1.196577e+07,8.784292e+06,5.949462e+08,5.963890e+08,1.599527e+08,4.318435e+08
9,1002,利润表,2017-12-31,6.761551e+09,4.018886e+09,8.834603e+07,3.866863e+07,4.231062e+08,1.877875e+07,4.561828e+07,2.164937e+09,2.165782e+09,5.645379e+08,1.564239e+09


In [7]:
#查看出现较多的时间
data['日期'].value_counts()

2017-09-30    3201
2017-06-30    3146
2017-12-31    3133
2018-03-31    3104
2018-06-30    3065
2018-09-30    3031
2016-12-31     202
2017-03-31     176
2016-09-30     169
2016-06-30     138
2015-12-31      89
2016-03-31      77
2018-12-31      46
2014-12-31       5
16-06-30         4
15-12-31         4
15-06-30         4
16-03-31         4
15-09-30         4
2013-12-31       3
15-03-31         3
14-12-31         2
14-09-30         2
16-09-30         2
16-12-31         2
2015-09-30       1
2015-06-30       1
17-03-31         1
2011-12-31       1
2012-12-31       1
Name: 日期, dtype: int64

In [8]:
#补全时间，规整时间的格式
pattern='\d\d\d\d-\d\d-\d\d'
for i in range(len(data)):
    if re.findall(pattern,data.iloc[i,2])==[]:
        data.iloc[i,2]='20'+data.iloc[i,2]

In [9]:
#去除企业编号与日期值同时相同的行，保留第一次出现的行
data.drop_duplicates(subset=['企业编号','日期'],keep='first',inplace=True)

In [10]:
#再次查看出现较多的时间
data['日期'].value_counts()

2017-09-30    2982
2017-12-31    2980
2018-06-30    2974
2018-09-30    2973
2018-03-31    2957
2017-06-30    2928
2016-12-31     163
2017-03-31     136
2016-09-30     131
2016-06-30     109
2015-12-31      78
2016-03-31      66
2018-12-31      45
2014-12-31       7
2015-09-30       5
2015-06-30       5
2015-03-31       3
2013-12-31       3
2014-09-30       2
2011-12-31       1
2012-12-31       1
Name: 日期, dtype: int64

In [11]:
#将数量较少的日期全部丢弃
timelist=list(['2017-09-30','2017-12-31','2018-06-30','2018-09-30','2017-06-30','2018-03-31'])
data=data[data['日期'].isin(timelist)]
data

,企业编号,标题,日期,营业收入(元),营业成本(元),销售费用(元),财务费用(元),管理费用(元),资产减值损失(元),投资收益(元),营业利润(元),利润总额(元),所得税(元),归属母公司所有者净利润(元)
0,1001,利润表,2018-09-30,1.271090e+11,NaN,NaN,NaN,3.042100e+10,4.407200e+10,1.111600e+10,5.091800e+10,5.089300e+10,7.229000e+09,4.303400e+10
1,1001,利润表,2018-06-30,8.225600e+10,NaN,NaN,NaN,1.997100e+10,2.692900e+10,9.382000e+09,3.416400e+10,3.416100e+10,5.261000e+09,2.843200e+10
2,1001,利润表,2018-03-31,3.962900e+10,NaN,NaN,NaN,9.680000e+09,1.217300e+10,3.696000e+09,1.710500e+10,1.710600e+10,2.647000e+09,1.426600e+10
3,1001,利润表,2017-12-31,1.686190e+11,NaN,NaN,NaN,4.104700e+10,5.528500e+10,1.358500e+10,7.027500e+10,6.982800e+10,1.482600e+10,5.404600e+10
4,1001,利润表,2017-09-30,1.254710e+11,NaN,NaN,NaN,2.927500e+10,4.014500e+10,8.391000e+09,5.462100e+10,5.474300e+10,1.234700e+10,4.130500e+10
5,1001,利润表,2017-06-30,8.404500e+10,NaN,NaN,NaN,1.923600e+10,2.717000e+10,6.138000e+09,3.667100e+10,3.675100e+10,8.229000e+09,2.761800e+10
6,1002,利润表,2018-09-30,5.803395e+09,4.145927e+09,2.152162e+07,7.248514e+07,3.372786e+08,1.189036e+07,3.585955e+07,1.247170e+09,1.252524e+09,3.128934e+08,9.146334e+08
7,1002,利润表,2018-06-30,3.720616e+09,2.524410e+09,1.605859e+07,3.204614e+07,1.885846e+08,3.181400e+07,1.778522e+07,9.519288e+08,9.549638e+08,2.391728e+08,7.008572e+08
8,1002,利润表,2018-03-31,1.723799e+09,1.039236e+09,1.367522e+07,-5.158956e+06,6.735945e+07,1.196577e+07,8.784292e+06,5.949462e+08,5.963890e+08,1.599527e+08,4.318435e+08
9,1002,利润表,2017-12-31,6.761551e+09,4.018886e+09,8.834603e+07,3.866863e+07,4.231062e+08,1.877875e+07,4.561828e+07,2.164937e+09,2.165782e+09,5.645379e+08,1.564239e+09


In [12]:
data.sort_values(by=['企业编号','日期'],inplace=True)

D:\ANACONDA2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
#查看排序结果是否达到目标效果
data

,企业编号,标题,日期,营业收入(元),营业成本(元),销售费用(元),财务费用(元),管理费用(元),资产减值损失(元),投资收益(元),营业利润(元),利润总额(元),所得税(元),归属母公司所有者净利润(元)
5,1001,利润表,2017-06-30,8.404500e+10,NaN,NaN,NaN,1.923600e+10,2.717000e+10,6.138000e+09,3.667100e+10,3.675100e+10,8.229000e+09,2.761800e+10
4,1001,利润表,2017-09-30,1.254710e+11,NaN,NaN,NaN,2.927500e+10,4.014500e+10,8.391000e+09,5.462100e+10,5.474300e+10,1.234700e+10,4.130500e+10
3,1001,利润表,2017-12-31,1.686190e+11,NaN,NaN,NaN,4.104700e+10,5.528500e+10,1.358500e+10,7.027500e+10,6.982800e+10,1.482600e+10,5.404600e+10
2,1001,利润表,2018-03-31,3.962900e+10,NaN,NaN,NaN,9.680000e+09,1.217300e+10,3.696000e+09,1.710500e+10,1.710600e+10,2.647000e+09,1.426600e+10
1,1001,利润表,2018-06-30,8.225600e+10,NaN,NaN,NaN,1.997100e+10,2.692900e+10,9.382000e+09,3.416400e+10,3.416100e+10,5.261000e+09,2.843200e+10
0,1001,利润表,2018-09-30,1.271090e+11,NaN,NaN,NaN,3.042100e+10,4.407200e+10,1.111600e+10,5.091800e+10,5.089300e+10,7.229000e+09,4.303400e+10
11,1002,利润表,2017-06-30,3.213662e+09,1.945463e+09,3.555172e+07,-3.117221e+06,1.886513e+08,2.316763e+06,1.882209e+07,1.050402e+09,1.051206e+09,2.647543e+08,7.803330e+08
10,1002,利润表,2017-09-30,4.918489e+09,3.002666e+09,5.189270e+07,-2.303142e+06,2.845893e+08,8.285495e+06,3.722918e+07,1.607031e+09,1.577293e+09,3.972435e+08,1.198716e+09
9,1002,利润表,2017-12-31,6.761551e+09,4.018886e+09,8.834603e+07,3.866863e+07,4.231062e+08,1.877875e+07,4.561828e+07,2.164937e+09,2.165782e+09,5.645379e+08,1.564239e+09
8,1002,利润表,2018-03-31,1.723799e+09,1.039236e+09,1.367522e+07,-5.158956e+06,6.735945e+07,1.196577e+07,8.784292e+06,5.949462e+08,5.963890e+08,1.599527e+08,4.318435e+08


In [14]:
data=data.set_index(['企业编号','日期'])
data=data.unstack().reset_index().set_index('企业编号').T.drop('标题')

In [15]:
data=data.reindex(columns=list(range(1001,4001)))
data.to_pickle('上市信息财务信息-利润表.pickle')
data

企业编号                              1001         1002         1003         1004  \
               日期                                                               
营业收入(元)        2017-06-30   8.4045e+10  3.21366e+09  8.14589e+09  1.25497e+09   
               2017-09-30  1.25471e+11  4.91849e+09  1.29964e+10  1.99218e+09   
               2017-12-31  1.68619e+11  6.76155e+09  1.83009e+10  2.76646e+09   
               2018-03-31   3.9629e+10   1.7238e+09  2.45778e+09   7.7753e+08   
               2018-06-30   8.2256e+10  3.72062e+09  5.78917e+09  1.55212e+09   
               2018-09-30  1.27109e+11  5.80339e+09  9.24283e+09  2.33654e+09   
营业成本(元)        2017-06-30          NaN  1.94546e+09  7.24443e+09  5.78449e+08   
               2017-09-30          NaN  3.00267e+09  1.15687e+10  9.42475e+08   
               2017-12-31          NaN  4.01889e+09  1.63431e+10   1.3396e+09   
               2018-03-31          NaN  1.03924e+09   2.0959e+09  3.72713e+08   
               2018-06-30          NaN  2.52441e+09   5.0713e+09  7.28342e+08   
               2018-09-30          NaN  4.14593e+09  8.19203e+09  1.09525e+09   
销售费用(元)        2017-06-30          NaN  3.55517e+07  4.24572e+08  4.19261e+07   
               2017-09-30          NaN  5.18927e+07  7.65363e+08  6.02727e+07   
               2017-12-31          NaN   8.8346e+07   1.0186e+09  9.47737e+07   
               2018-03-31          NaN  1.36752e+07  1.22953e+08  1.04714e+07   
               2018-06-30          NaN  1.60586e+07  2.72415e+08  2.76326e+07   
               2018-09-30          NaN  2.15216e+07  4.17515e+08  4.60654e+07   
财务费用(元)        2017-06-30          NaN -3.11722e+06 -2.01935e+07  5.86675e+07   
               2017-09-30          NaN -2.30314e+06 -7.60198e+06  1.01251e+08   
               2017-12-31          NaN  3.86686e+07 -1.14174e+07  1.41558e+08   
               2018-03-31          NaN -5.15896e+06 -3.21156e+06  3.73582e+07   
               2018-06-30          NaN  3.20461e+07 -1.60898e+07  7.10632e+07   
               2018-09-30          NaN  7.24851e+07 -1.94951e+07  1.08634e+08   
管理费用(元)        2017-06-30   1.9236e+10  1.88651e+08  5.77859e+08  6.32755e+07   
               2017-09-30   2.9275e+10  2.84589e+08  8.81906e+08  8.37301e+07   
               2017-12-31   4.1047e+10  4.23106e+08  1.10121e+09  1.10857e+08   
               2018-03-31     9.68e+09  6.73595e+07  1.99652e+08  2.04399e+07   
               2018-06-30   1.9971e+10  1.88585e+08   4.1144e+08  4.10844e+07   
               2018-09-30   3.0421e+10  3.37279e+08  6.46652e+08  6.16354e+07   
...                                ...          ...          ...          ...   
投资收益(元)        2017-06-30    6.138e+09  1.88221e+07  2.65996e+08  2.42543e+06   
               2017-09-30    8.391e+09  3.72292e+07  3.80402e+08  5.51164e+06   
               2017-12-31   1.3585e+10  4.56183e+07  4.44671e+08  8.66791e+06   
               2018-03-31    3.696e+09  8.78429e+06  1.07747e+08  2.81326e+06   
               2018-06-30    9.382e+09  1.77852e+07  2.93019e+08  3.96618e+06   
               2018-09-30   1.1116e+10  3.58596e+07  4.56705e+08  6.66262e+06   
营业利润(元)        2017-06-30   3.6671e+10   1.0504e+09  5.77859e+06  3.93754e+08   
               2017-09-30   5.4621e+10  1.60703e+09 -1.67333e+08  6.23709e+08   
               2017-12-31   7.0275e+10  2.16494e+09  -8.9908e+07  8.27209e+08   
               2018-03-31   1.7105e+10  5.94946e+08   1.3979e+08  2.68932e+08   
               2018-06-30   3.4164e+10  9.51929e+08  2.90976e+08  5.50701e+08   
               2018-09-30   5.0918e+10  1.24717e+09  3.84793e+08  8.21564e+08   
利润总额(元)        2017-06-30   3.6751e+10  1.05121e+09      -133306   3.9342e+08   
               2017-09-30   5.4743e+10  1.57729e+09 -1.96006e+08   6.3644e+08   
               2017-12-31   6.9828e+10  2.16578e+09 -1.18883e+08  8.49004e+08   
               2018-03-31   1.7106e+10  5.96389e+08  1.39051e+08  2.68868e+08   
               2018-06-30   3.4